In [ ]:
# 1. Simple mode:
# - Product Recommendation Based on Popularity by Month (and/or by Location) for New Customers
# - Product Recommendation Based on Highly Rated Categories

### Prepration:
Create utility matrix between users and products. This could be any relation between users and products such as click or not, watching time, rating... Our objective is to fill missing items. Note that these items generally very large compared to the available values. 

### Approaches: 
https://github.com/Chinmayrane16/Recommender-Systems-with-Collaborative-Filtering-and-Deep-Learning-Techniques/blob/master/Recommendation%20Systems%20with%20CF%20and%20DL%20Techniques.ipynb

1. Content-based recommendation system

Based on feature values of products. **For each user**, we create a ML model to learn from the input feature values to the output (scores from utility matrix).

2. Collaborative filtering: take the relationship between users into account
- Memory-based filtering: utility matrix -> normalize (average subtraction), combine with similarity matrix (average of k-nearest users)
  + User-user:  similarity is calculated between users --> "People like you also like", "You might also like"
  + Item-item: similarity is calculated between items --> "Users who bought this might also like this" (similar items). Much faster than user-user, and secondly user profiles changes quickly and the entire system model has to be recomputed, whereas item's average ratings doesn't change that quickly, and this leads to more stable rating distributions in the model, so the model doesn't have to be rebuilt as often.
  + Similarity: Jaccard, cosine (most common), Euclide, pearson
- Model-based filtering: Basically, we compress user-item matrix into a low dimension matrix. We use techniques like SVD which is a low-rank factorization method, PCA which is used for dimensionaliry reduction etc.
  + K-Means: we could use K-Means to group our users to multiple clusters, then making recommendation based on their's clusters (the average or the centeroid).
  + SVD
  + KNN
  + NMF
- NMF using Deep Learning

The concept behind matrix factorization models is that the preferences of a user can be determined by a small number of hidden factors. And these are called as Embeddings. There are two types of architectures:
  + Architecture 1: dot product Embeddings - we embedding user and item matrix separately, then use dot product to combine these information to create features
  + Architecture 2: concatenation Embeddings - we concatenate user and item matrix after using Embedding to create features

### Evaluation
Using RMSE, compare with the baseline: missing values equal average for each users.

In [ ]:
# 2. Alternating Least Squares: customer with nearest distance?
# https://github.com/Nikhilkohli1/Olist-Marketing-Analytics/blob/master/5.%20Recommendation%20System/Jupyter%20Notebooks/Olist_Alternating_Least_Squares_SVD_.ipynb
# Recommending top 10 highly coorelated products in sequence

In [ ]:
# Create models --> save to models
# Deploy on Heroku / Flask (local only)

# Before Lunar New Year (01/02/2024)

Notes

Recommender systems are algorithms aimed at suggesting relevant items to users (items being movies to watch, text to read, products to buy or anything else depending on industries).

Recommender systems are really critical in some industries as they can generate a huge amount of income when they are efficient or also be a way to stand out significantly from competitors. As a proof of the importance of recommender systems, we can mention that, a few years ago, Netflix organised a challenge (the "Netflix prize") where the goal was to produce a recommender system that performs better than its own algorithm with a prize of 1 million dollars to win.

Image

We will be using below approach to Recommend products - 
 
1. Recommendation for Potential Loyalists and Other - 
ALS (Alternating Least Squares) is an implicit recommendation algorithm to make a recommendation of products and product categories to the users. ALS is an iterative optimization process where for every iteration it tries to arrive closer and closer to a factorized representation of the original data.

2. Recommendation for Loyal Customers who buys Products Frequently and Recently -
Customer2Vec is a good method for these kinds of users. As these customers usually repeat their orders and buys frequently. We learn a semantic representation for Customers using their previous purchase history.

We learn good semantic representations for customers (users) from transactional data using doc2vec. Each customer is a document, orders are sentences, and products are words. We also compare Customer2Vec with the baseline

3. For Big Spenders - 
We use Upselling to recommend products to Big Spenders and VVIP customers using Highest Rated and High end prodcuts. 

4. For new Customers - 
We show Popular Products by current month for new customers. 